# SyftBox SDK Development Testing

This notebook is for testing and validating the SyftBox SDK during development.

In [1]:
%%capture output --no-stderr

# Install the SDK in development mode (run this once)
!uv pip install nest_asyncio

In [2]:
# Import the SDK
import syft_nsai_sdk as sdk
import nest_asyncio
import httpx
import logging
import os

In [3]:
# import client
from syft_nsai_sdk import SyftBoxClient
from syft_nsai_sdk.utils.logger import get_logger
from IPython.display import display, HTML, Markdown

In [4]:
# define async
nest_asyncio.apply()

# initialize logger
logger = get_logger()
logger.success("Logger initialized successfully!")

# Turn off debug logging for your SDK
logging.getLogger('syft_nsai_sdk').setLevel(logging.INFO)

2025-08-29 16:12:52 - syft_nsai_sdk - SUCCESS - Logger initialized successfully!


In [5]:
# initialize client
client = SyftBoxClient()
logger.info(f"Syftbox client created: {client}")

2025-08-29 16:12:53 - syft_nsai_sdk.main - INFO - Found existing accounting credentials in environment
2025-08-29 16:12:53 - syft_nsai_sdk.main - INFO - SyftBoxClient initialized for guest@syft.org
2025-08-29 16:12:53 - syft_nsai_sdk - INFO - Syftbox client created: <syft_nsai_sdk.main.SyftBoxClient object at 0x106561370>


In [6]:
# Get accounting env variables
# email = os.getenv('SYFTBOX_ACCOUNTING_EMAIL')
# password = os.getenv('SYFTBOX_ACCOUNTING_PASSWORD')
# logger.info(f"SYFTBOX_ACCOUNTING_EMAIL: {email}")
# logger.info(f"SYFTBOX_ACCOUNTING_PASSWORD: {bool(password)}")

# # Setup accounting for paid models
# await client.setup_accounting(email, password)

In [7]:
# STEP 2: Discover models explicitly 
models = client.discover_models(service_type="chat")
print(f"Found {len(models)} models")

2025-08-29 16:12:55 - syft_nsai_sdk.discovery.scanner - INFO - Found 12 models across 256 datasites
2025-08-29 16:12:55 - syft_nsai_sdk.discovery.scanner - INFO - Cached 12 models from 10 owners
2025-08-29 16:13:01 - syft_nsai_sdk.services.health - INFO - Batch health check completed in 5.85s for 8 models
2025-08-29 16:13:01 - syft_nsai_sdk.main - INFO - Discovered 8 models (health_check=True)


Found 8 models


In [8]:
print(f"{'Name':<20} {'Owner':<25} {'Tags':<25} {'Services':<25}")
print("-" * 100)
for model in models:
    enabled_services = [s.type.value for s in model.services if s.enabled]
    tags_str = ", ".join(model.tags[:2])
    services_str = ", ".join(enabled_services)
    print(f"{model.name:<20} {model.owner:<25} {tags_str:<25} {services_str:<25}")

Name                 Owner                     Tags                      Services                 
----------------------------------------------------------------------------------------------------
keelan-test-3        kj@kj.dev                 test                      chat, search             
pal-8                afdolriski@gmail.com                                chat, search             
carl-model           callis@openmined.org      Chat, Search              chat, search             
carl-free            callis@openmined.org      chat, search              chat, search             
claude-sonnet-3.5    aggregator@openmined.org  claude, anthropic         chat                     
deepseek-v3          aggregator@openmined.org  deepseek, opensource      chat                     
test                 irina@openmined.org       test                      chat, search             
sota-model           callisezenwaka@barestripe.com chat, search              chat, search             


In [9]:
print(await client.get_account_info())

{'email': 'callis@openmined.org', 'balance': 115.3, 'currency': 'USD', 'account_id': 'cme1ec567000aqn3z9vu7q3st', 'created_at': None}


In [29]:
# Create conversation (enhanced with owner support)
conversation = client.create_conversation(
    model_name="claude-sonnet-3.5",
    owner="aggregator@openmined.org",
)

2025-08-29 16:14:52 - syft_nsai_sdk.main - INFO - Discovered 1 models (health_check=False)


In [30]:
# Clear history when needed - defaults context messages = []
conversation.clear_history()

# Set context - default is 2 previous responses
# conversation.set_max_exchanges(1)

In [31]:
# Each message remembers full context
response1 = await conversation.send_message("What is SyftBox?")

✓ Response received                                                             



In [24]:
# After each response, you can check the conversation state
# logger.info(f"Messages count: {len(conversation.messages)}")
# logger.info(f"Context length: {len(conversation._build_context_prompt('test'))}")

In [32]:
logger.info(f"Response:\n\n{response1}")

2025-08-29 16:15:02 - syft_nsai_sdk - INFO - Response:

SyftBox is a project from OpenMined focused on creating a privacy-preserving platform for AI model deployment. It allows data scientists and AI researchers to train and deploy machine learning models on sensitive data without actually seeing or accessing the raw data. 

SyftBox implements privacy-enhancing technologies like federated learning, differential privacy, and secure multi-party computation, enabling organizations to collaborate on AI projects while maintaining data privacy and security compliance.

The technology is designed to help organizations overcome data privacy challenges in AI development, particularly in regulated industries like healthcare and finance where data privacy is critical.


In [33]:
# second conversation
response2 = await conversation.send_message("How does it work?")  # Remembers first question

✓ Response received                                                             



In [34]:
logger.info(f"Response:\n\n{response2}")

2025-08-29 16:15:14 - syft_nsai_sdk - INFO - Response:

SyftBox works by creating a secure framework for privacy-preserving AI operations through several key mechanisms:

1. **Federated Architecture**: Instead of centralizing sensitive data, SyftBox allows models to be sent to where the data resides. The models are trained locally on each data node, and only model updates (not the raw data) are shared back.

2. **Privacy-Preserving Computations**: SyftBox implements several techniques:
   - **Secure Multi-Party Computation (SMPC)** for calculations across multiple parties without revealing individual inputs
   - **Differential Privacy** to add calibrated noise to outputs, protecting individual data points
   - **Homomorphic Encryption** allowing computations on encrypted data

3. **Domain-Specific Language**: SyftBox provides a Python-based DSL that allows data scientists to write code that's automatically translated into privacy-preserving operations.

4. **Node Network**: It establis

In [35]:
# third conversation
response3 = await conversation.send_message("Give an example")    # Remembers both previous

✓ Response received                                                             



In [36]:
logger.info(f"Response:\n\n{response3}")

2025-08-29 16:15:26 - syft_nsai_sdk - INFO - Response:

Let me provide a concrete example of how SyftBox might be used in a real-world scenario:

### Healthcare Collaboration Example

**Scenario**: Three hospitals want to collaborate on developing a better diagnostic model for rare diseases, but cannot share patient data due to HIPAA regulations and privacy concerns.

**Step 1: Setup**
- Each hospital installs SyftBox and creates a data node containing their patient records
- They define privacy policies (e.g., differential privacy parameters, what data fields can be accessed)

**Step 2: Model Development**
```python
# Researcher code (simplified example)
import syft as sy

# Connect to hospital data nodes
hospital_a = sy.login(url="hospital-a.org")
hospital_b = sy.login(url="hospital-b.org")
hospital_c = sy.login(url="hospital-c.org")

# Define model architecture
model = sy.LogisticRegression()

# Create federated dataset pointer (no actual data is transferred)
federated_data = sy.fed

In [20]:
# Get conversation summary
# summary = conversation.get_conversation_summary()
# logger.info(f"Total messages: {summary['total_messages']}")

2025-08-29 16:06:42 - syft_nsai_sdk - INFO - Total messages: 4
